In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
# Load data
data = pd.read_csv("customer_churn_dataset-testing-master.csv")
data.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,1,22,Female,25,14,4,27,Basic,Monthly,598,9,1
1,2,41,Female,28,28,7,13,Standard,Monthly,584,20,0
2,3,47,Male,27,10,2,29,Premium,Annual,757,21,0
3,4,35,Male,9,12,5,17,Premium,Quarterly,232,18,0
4,5,53,Female,58,24,9,2,Standard,Annual,533,18,0


In [3]:
data2 = data.drop(["CustomerID",'Subscription Type','Contract Length'],axis=1)
data2.head(1)

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
0,22,Female,25,14,4,27,598,9,1


In [4]:
data.isnull().sum()

CustomerID           0
Age                  0
Gender               0
Tenure               0
Usage Frequency      0
Support Calls        0
Payment Delay        0
Subscription Type    0
Contract Length      0
Total Spend          0
Last Interaction     0
Churn                0
dtype: int64

In [5]:
# Encoding
encode = LabelEncoder()
data2['Gender'] = encode.fit_transform(data2["Gender"])

In [6]:
data2

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Churn
0,22,0,25,14,4,27,598,9,1
1,41,0,28,28,7,13,584,20,0
2,47,1,27,10,2,29,757,21,0
3,35,1,9,12,5,17,232,18,0
4,53,0,58,24,9,2,533,18,0
...,...,...,...,...,...,...,...,...,...
64369,45,0,33,12,6,21,947,14,1
64370,37,1,6,1,5,22,923,9,1
64371,25,1,39,14,8,30,327,20,1
64372,50,0,18,19,7,22,540,13,1


In [7]:
X = data2.drop('Churn',axis=1)
Y = data2['Churn']

In [1]:
X

NameError: name 'X' is not defined

In [8]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,random_state=3,test_size=0.2)

In [9]:
scale = StandardScaler()
x_train_scale = scale.fit_transform(x_train)
x_test_scale = scale.fit_transform(x_test)

In [10]:
pickle.dump(encode,open("encode.pkl","wb"))
pickle.dump(scale,open("scale.pkl","wb"))

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.callbacks import EarlyStopping , TensorBoard

In [15]:
model = Sequential([
    Dense(64,activation='relu',input_shape = (x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])


C:\Users\LAXMIKANT\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,689 (10.50 KB)

 Trainable params: 2,689 (10.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [58]:
import datetime
log_dir = 'logs/fit'
tensorflow_callback = TensorBoard(log_dir = log_dir ,histogram_freq=1)

In [59]:
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

In [60]:
history = model.fit(
    x_train_scale,y_train,validation_data=(x_test_scale,y_test), epochs=5,
    callbacks = [tensorflow_callback,early_stopping_callback]
)

Epoch 1/5
1610/1610 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9658 - loss: 0.0653 - val_accuracy: 0.9577 - val_loss: 0.0724
Epoch 2/5
1610/1610 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9647 - loss: 0.0648 - val_accuracy: 0.9617 - val_loss: 0.0728
Epoch 3/5
1610/1610 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9642 - loss: 0.0679 - val_accuracy: 0.9640 - val_loss: 0.0686
Epoch 4/5
1610/1610 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9628 - loss: 0.0674 - val_accuracy: 0.9606 - val_loss: 0.0699
Epoch 5/5
1610/1610 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9645 - loss: 0.0658 - val_accuracy: 0.9567 - val_loss: 0.0733


In [69]:
model.save('model.h5')